In [4]:
!pip install sentence-transformers
!pip install -q langchain
!pip install -q faiss-cpu
!pip install -q pypdf
!pip install openai
!pip install python-dotenv
!pip install tqdm
!pip install -U langchain-community
!pip install -qU langchain-openai
!pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.2 MB/s eta 0:00:00


In [48]:
!pip install minsearch

In [2]:
cd '/content/drive/MyDrive/Courses/AI/Lecture14'

/content/drive/MyDrive/Courses/AI/Lecture14


In [6]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [8]:
import pandas as pd

ground_truth_file = "ground-truth-data.csv"

df_ground_truth = pd.read_csv(ground_truth_file)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [9]:
ground_truth[10]

{'question': 'Are sessions recorded if I am unable to attend?',
 'course': 'machine-learning-zoomcamp',
 'document': '5170565b'}

In [10]:
doc_idx = {d['id']: d for d in documents}
doc_idx['5170565b']['text']

'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.'

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'course', 'id'])

In [12]:
from uuid import uuid4
from langchain_core.documents import Document

docs = []
idx = []

for doc in documents:
    document = Document(page_content = doc["text"] + doc["question"], metadata={'id': doc['id'], "section": doc["section"], "course": doc["course"]})
    idx.append(doc["id"])
    docs.append(document)

In [13]:
from langchain.vectorstores import FAISS

In [14]:
# Create the FAISS index with all embeddings and metadata in batch
db = FAISS.from_documents(docs, hf)

In [15]:
# Optional: Save the index for later use
db.save_local("faiss_index")

In [16]:
#Similarity search
#https://python.langchain.com/v0.2/docs/integrations/vectorstores/faiss/
results = db.similarity_search(
    "Are sessions recorded if I miss one?",
    filter={"course": "machine-learning-zoomcamp"},
)

for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.What if I miss a session? [{'id': '5170565b', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}]
* The course videos are pre-recorded, you can start watching the course right now.
We will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.
You can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.Is it going to be live? When? [{'id': '39fda9f0', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}]
* We won’t re-record the course videos. The focus of the course and the skills we want to teach remained the same, and the videos are still up-to-date.
If you haven’t taken part in the prev

In [19]:
results[0].page_content

'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.What if I miss a session?'

In [22]:
from tqdm.notebook import tqdm

In [20]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [57]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = db.similarity_search(
        q['question'],
        filter={"course": q['course']},
    )
    relevance = [d.metadata['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1858 [00:00<?, ?it/s]

In [59]:
relevance_total

[[False, False, False, False],
 [False, True, False, False],
 [False, True, False, False],
 [False, False, False, True],
 [False, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [False, True, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [False, False, False, True],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, False, False, False],
 [True, 

In [40]:
example = [
    [True, False, False, False, False], # 1,
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [False, False, True, False, False],  # 1/3
    [False, False, False, False, False], # 0
]

# 1 => 1
# 2 => 1 / 2 = 0.5
# 3 => 1 / 3 = 0.3333
# 4 => 0.25
# 5 => 0.2
# rank => 1 / rank
# none => 0

In [42]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [43]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [44]:
hit_rate(example)

0.5833333333333334

In [45]:
mrr(example)

0.5277777777777778



- hit-rate (recall)
- Mean Reciprocal Rank (mrr)



In [58]:
hit_rate(relevance_total), mrr(relevance_total)

(0.9327233584499461, 0.8549515608180835)

In [49]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

In [50]:
def minsearch_search(query, course):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

In [62]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1858 [00:00<?, ?it/s]

In [63]:
hit_rate(relevance_total), mrr(relevance_total)

(0.7992465016146394, 0.6907696447793331)

In [64]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d.metadata['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [65]:
evaluate(ground_truth, lambda q: db.similarity_search(q['question'],filter={"course": q['course']}))

  0%|          | 0/1858 [00:00<?, ?it/s]

{'hit_rate': 0.9327233584499461, 'mrr': 0.8549515608180835}